In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

True

In [7]:
model = ChatOpenAI(model="gpt-4-turbo-preview")

prompt = ChatPromptTemplate.from_template("{topic}에 대한 짧은 농담을 해줘")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [8]:
chain.invoke({"topic": "아이스크림"})

'아이스크림이 은행에 갔어요. 왜냐하면 자기 계좌를 녹이고 싶어서요!'

In [9]:
prompt_value = prompt.invoke({"topic": "아이스크림"})

prompt_value

ChatPromptValue(messages=[HumanMessage(content='아이스크림에 대한 짧은 농담을 해줘')])

In [11]:
print(prompt_value.to_messages())

print(prompt_value.to_string())

[HumanMessage(content='아이스크림에 대한 짧은 농담을 해줘')]
Human: 아이스크림에 대한 짧은 농담을 해줘


In [12]:
message = model.invoke(prompt_value)

message

AIMessage(content='아이스크림에게 무슨 일이 있어도 녹지 않겠다고 다짐했더니, 아이스크림이 말하길 "그건 좀 차가운 태도네!"', response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 28, 'total_tokens': 92}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_13c70b9f70', 'finish_reason': 'stop', 'logprobs': None})

In [13]:
output_parser.invoke(message)

'아이스크림에게 무슨 일이 있어도 녹지 않겠다고 다짐했더니, 아이스크림이 말하길 "그건 좀 차가운 태도네!"'

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

In [9]:
model = ChatOpenAI(model="gpt-4-turbo-preview")

vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "harrison worked at kensho",
        "bears like to eat honey",
    ],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """반드시 주어진 맥락에 근거하여 질문에 대해 답변하라:
{context}

질문: {question}"""

prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
)

chain = setup_and_retrieval | prompt | model | output_parser

In [10]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'